In [ ]:
%load_ext autoreload
%autoreload 2

%aimport src.embeddings.batcher
%aimport src.lstm.batcher
%aimport src.lstm.lstm
%aimport src.embeddings.embeddings
%aimport src.dataset

In [ ]:
import os
import src.util as util
from six.moves.urllib.request import urlretrieve
from src.dataset import DatasetWithEmbeddings
from src.dataset import Dataset
from src.embeddings.embeddings import Embeddings

from src.lstm.batcher import BatchConfig
from src.lstm.batcher import BatchGenerator
from src.lstm.lstm import LSTMModel
from src.lstm.lstm import LSTMModelConfig

In [ ]:
validation_size = 1000
filename = "data/text8.zip"

In [ ]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + 'text8.zip', filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

maybe_download(filename, 31344016)

Embeddings params:

In [ ]:
num_sampled = 64
embedding_length = 32

Read and preprocess data from embeddings

In [ ]:
text = util.read_data(filename)

In [ ]:
dataset = Dataset.create_from_text(text)

In [ ]:
batch_config = src.embeddings.batcher.BatcherConfig(batch_size=128, num_skips=1, skip_window=2)
batcher = src.embeddings.batcher.Batcher(dataset, batch_config)

embeddings = Embeddings(batcher=batcher,
                        embedding_length=embedding_length,
                        vocabulary_size=dataset.get_bigrams_len(),
                        num_sampled=num_sampled)

Find embeddings:

In [ ]:
training_epochs = 10001
final_embeddings = embeddings.build_embeddings(training_epochs)

Prepare data for LSTM

In [ ]:
train_dataset, validation_dataset = dataset.train_validation_split(validation_size)

In [ ]:
dataset_with_embeddings = DatasetWithEmbeddings(train_dataset, final_embeddings)

LSTM params:

In [ ]:
batch_size = 64
num_unrollings = 10

In [ ]:
batch_config = BatchConfig(batch_size=batch_size, num_unrollings=num_unrollings)
batcher = BatchGenerator(dataset_with_embeddings, batch_config)

batch, labels = batcher.batch()

Train LSTM

In [ ]:
save_model_to = "model/model.ckpt"
lstm_model_config = LSTMModelConfig(dataset_with_embeddings.get_embedding_size(), 
                                              dataset_with_embeddings.get_bigrams_len(), 
                                              batch_config)
lstm = LSTMModel(lstm_model_config, save_model_to)

In [ ]:
lstm.train(batcher, 5000)

Evaluate model:

In [ ]:
print lstm.generate_text(['aa', ' b', 'be', ' m'], dataset_with_embeddings, 24)

In [ ]:
validation_dataset_with_embeddings = DatasetWithEmbeddings(validation_dataset, final_embeddings)
print lstm.measure_validation_perplexity(validation_dataset_with_embeddings)